In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("uber.csv")

In [3]:
df.shape

(200000, 8)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ride_id            200000 non-null  int64  
 1   fare_amount        200000 non-null  float64
 2   pickup_datetime    200000 non-null  object 
 3   pickup_longitude   200000 non-null  float64
 4   pickup_latitude    200000 non-null  float64
 5   dropoff_longitude  199999 non-null  float64
 6   dropoff_latitude   199999 non-null  float64
 7   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 12.2+ MB


In [5]:
pd.to_datetime(df['pickup_datetime'])

0        2015-05-07 19:52:06+00:00
1        2009-07-17 20:04:56+00:00
2        2009-08-24 21:45:00+00:00
3        2009-06-26 08:22:21+00:00
4        2014-08-28 17:47:00+00:00
                    ...           
199995   2012-10-28 10:49:00+00:00
199996   2014-03-14 01:09:00+00:00
199997   2009-06-29 00:42:00+00:00
199998   2015-05-20 14:56:25+00:00
199999   2010-05-15 04:08:00+00:00
Name: pickup_datetime, Length: 200000, dtype: datetime64[ns, UTC]

In [6]:
# df['pickup_datetime'].astype("datetime64[ns]")

In [7]:
df.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [8]:
df['fare_amount'].mean()

11.359955250000002

In [9]:
# df['dropoff_latitude'].value_counts()

In [10]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
      # Earth's radius in kilometers
    EARTH_RADIUS = 6371.0  
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return EARTH_RADIUS * c

In [11]:
df_new = (
    df.assign(
    haversine_dist=lambda df_: df_.apply(
        lambda row: haversine(
            row['pickup_latitude'], row['pickup_longitude'], 
            row['dropoff_latitude'], row['dropoff_longitude']
        ), 
        axis=1
        )
    )
)

In [12]:
df_new.haversine_dist.median()

2.1209923961833708

In [13]:
df_new.query("haversine_dist == 0.0").shape

(5632, 9)

In [14]:
df_new.query("haversine_dist == 0.0")['fare_amount'].mean()

11.585317826704546

In [15]:
df_new['fare_amount'].max()

499.0

In [16]:
df_new.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,haversine_dist
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1,1.683323
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1,2.457590
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1,5.036377
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3,1.661683
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5,4.475450


In [17]:
df_new.query("fare_amount == 499.0").haversine_dist

170081    0.00079
Name: haversine_dist, dtype: float64

In [18]:
(
    df_new
    .assign(
        pickup_datetime = lambda df_: (
            pd.to_datetime(df_.pickup_datetime)
        ),
        pickup_year=lambda df_: (
            df_.pickup_datetime.dt.year
        )
    )

    .query("pickup_year == 2014")
)

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,haversine_dist,pickup_year
4,17610152,16.00,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,4.475450,2014
6,48725865,24.50,2014-10-12 07:04:00+00:00,-73.961447,40.693965,-73.871195,40.774297,5,11.731015,2014
20,55085966,10.50,2014-02-18 14:26:00+00:00,-73.980022,40.745990,-74.003432,40.759667,1,2.490244,2014
26,38755863,5.00,2014-01-21 06:55:00+00:00,-73.957802,40.776372,-73.957422,40.782870,1,0.723253,2014
34,19277743,39.50,2014-06-04 06:49:00+00:00,-73.788080,40.642187,-73.865042,40.725997,4,11.356141,2014
...,...,...,...,...,...,...,...,...,...,...
199979,9699676,6.00,2014-06-08 21:38:43+00:00,-73.975209,40.752767,-73.963186,40.757905,2,1.162745,2014
199981,21553740,9.00,2014-06-02 21:51:50+00:00,-73.980445,40.761823,-73.954004,40.784407,1,3.356153,2014
199982,13096190,57.33,2014-08-06 11:06:06+00:00,-73.969204,40.754771,-73.790351,40.643802,1,19.483047,2014
199994,3189201,12.00,2014-01-31 14:42:00+00:00,-73.983070,40.760770,-73.972972,40.754177,1,1.122878,2014


In [19]:
(
    df_new
    .assign(
        pickup_datetime = lambda df_: (
            pd.to_datetime(df_.pickup_datetime)
        ),
        pickup_year=lambda df_: (
            df_.pickup_datetime.dt.year
        ),
        pickup_month=lambda df_: (
            df_.pickup_datetime.dt.month_name()
        ),
        pickup_day=lambda df_: (
            df_.pickup_datetime.dt.day
        ),
        pickup_week_day=lambda df_: (
            df_.pickup_datetime.dt.day_name()
        )
    )

    .query("pickup_year == 2010 & pickup_month == 'September'")
    .groupby("pickup_week_day")
    .agg({
        "pickup_week_day" : "count"
    })
)

,pickup_week_day
pickup_week_day,
Friday,354
Monday,265
Saturday,362
Sunday,331
Thursday,457
Tuesday,322
Wednesday,391


In [20]:
df_new.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,haversine_dist
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1,1.683323
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1,2.457590
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1,5.036377
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3,1.661683
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5,4.475450


#### Data Cleaning function

In [21]:
def get_cleaned_data(df):
    return (
        df
        .assign(
            haversine_dist=lambda df_: df_.apply(
                lambda row: haversine(
                    row['pickup_latitude'], row['pickup_longitude'], 
                    row['dropoff_latitude'], row['dropoff_longitude']
                ), 
                axis=1
            ),
            pickup_datetime = lambda df_: (
                pd.to_datetime(df_.pickup_datetime)
            ),
            pickup_year=lambda df_: (
                df_.pickup_datetime.dt.year
            ),
            pickup_month=lambda df_: (
                df_.pickup_datetime.dt.month_name()
            ),
            pickup_day=lambda df_: (
                df_.pickup_datetime.dt.day
            ),
            pickup_week_day=lambda df_: (
                df_.pickup_datetime.dt.day_name()
            )
        )

    )

In [26]:
def training_features(df):
    # df.drop()
    return df[['passenger_count', 'haversine_dist', 'pickup_week_day', 'fare_amount']]

In [28]:
data = get_cleaned_data(df).pipe(training_features)
data.head()

,passenger_count,haversine_dist,pickup_week_day,fare_amount
0,1,1.683323,Thursday,7.5
1,1,2.457590,Friday,7.7
2,1,5.036377,Monday,12.9
3,3,1.661683,Friday,5.3
4,5,4.475450,Thursday,16.0


In [29]:
data.isnull().sum()

passenger_count    0
haversine_dist     1
pickup_week_day    0
fare_amount        0
dtype: int64

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['fare_amount']).values, df['fare_amount'].values)

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score

In [32]:
cat = Pipeline([
    ("imputer", SimpleImputer(strategy='most_frequent')),
    ("ohe", OneHotEncoder(dtype='int', handle_unknown='ignore'))
])

num = Pipeline([
    ("imputer", SimpleImputer(strategy='mean')),
    # ("scaler", StandardScaler())
])

In [33]:
pipe = ColumnTransformer([
    ("num", num, [0,1]),
    ("cat", cat, [2])
])

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

lr = LinearRegression()
rf = RandomForestRegressor(n_estimators=100)

In [35]:
lor_pipe = Pipeline([
        ("pre", pipe),
        ("model", LinearRegression())
])

rf_pipe = Pipeline([
        ("pre", pipe),
        ("model", RandomForestRegressor())
])

In [41]:
lor_pipe.fit(X_train, y_train)
rf_pipe.fit(X_train, y_train)
pass

In [43]:
lor_pipe

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  [0, 1]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(dtype='int',
                                                                                 handle_unknown='ignore'))]),
                                                  [2])])),
                ('model', LinearRegression())])

In [44]:
rf_pipe

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  [0, 1]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(dtype='int',
                                                                                 handle_unknown='ignore'))]),
                                                  [2])])),
                ('model', RandomForestRegressor())])

In [45]:
def get_r2_score(model, X, y):
    y_pred = model.predict(X)
    return r2_score(y, y_pred)

In [46]:
print(f"LR_train {get_r2_score(lor_pipe, X_train, y_train)}")
print(f"LR_test {get_r2_score(lor_pipe, X_test, y_test)}")

print(f"RF_train {get_r2_score(rf_pipe, X_train, y_train)}")
print(f"RF_test {get_r2_score(rf_pipe, X_test, y_test)}")

LR_train 0.0012552944822197265
LR_test 0.0012254874481594236
RF_train 0.9027372439205672
RF_test 0.6722844744234037
